In [ ]:
##################################################################################
finalise_outcome <- function(in_dat, sub_colname = "AOD_diff_case_t0"){
    rel_outcome = colnames(in_dat)[grepl(sub_colname, colnames(in_dat))]
    curr_levels <- sort(as.integer(unique(in_dat[, rel_outcome])))
    vec_here = factor(as.character(in_dat[, rel_outcome]), levels = curr_levels)
    return(vec_here)
}

get_bmi = function(in_path_cc, in_dat){
    res = in_dat
    CC_OA = fread(file = in_path_cc, sep = "\t", data.table = F)
    rownames(CC_OA) = CC_OA$ID_1

    bmi =  as.numeric(CC_OA[rownames(res), c("Body_mass_index_(BMI)")])

    return(bmi)
}

get_matrices <- function(in_path_cc, in_relcol_sub, in_outcome, in_path_sample_split, 
                         in_pred_mode = PRED_MODE){
    # check wether bmi and age and sex to be included
    is_bmi_used = (in_pred_mode != "prot_only")                             

    # load
    sample_split_obj = get(load(file = in_path_sample_split, verbose = T))

    # grep 
    p_mtx_traintest_ukbb = sample_split_obj$p_mtx_traintest
    pheno_train_test_ukbb = sample_split_obj$pheno_train_test

    p_mtx_val_ukbb = sample_split_obj$p_mtx_val
    pheno_val_ukbb = sample_split_obj$pheno_val

     # add bmi
    pheno_train_test_ukbb$bmi = get_bmi(in_path_cc = in_path_cc, in_dat = pheno_train_test_ukbb)
    pheno_val_ukbb$bmi = get_bmi(in_path_cc = in_path_cc, in_dat = pheno_val_ukbb)

    # make oa status a factor
    pheno_train_test_ukbb$oa_status =  finalise_outcome(in_dat = pheno_train_test_ukbb, sub_colname = in_relcol_sub)
    pheno_val_ukbb$oa_status =  finalise_outcome(in_dat = pheno_val_ukbb, sub_colname = in_relcol_sub)

    # keep samples with bmi information; add bmi to feature matrix
    if(is_bmi_used){
        # define pheno
        pheno_train_test_ukbb = pheno_train_test_ukbb[!is.na(pheno_train_test_ukbb$bmi), ]
        pheno_val_ukbb = pheno_val_ukbb[!is.na(pheno_val_ukbb$bmi), ]
        
        # define features
        p_mtx_traintest_ukbb = p_mtx_traintest_ukbb[rownames(pheno_train_test_ukbb), ]
        p_mtx_val_ukbb = p_mtx_val_ukbb[rownames(pheno_val_ukbb), ]

        # add age sex bmi to features
        p_mtx_traintest_ukbb = cbind(p_mtx_traintest_ukbb, pheno_train_test_ukbb[,c("bmi", "Age_at_recruitment", "Sex")])
        p_mtx_val_ukbb = cbind(p_mtx_val_ukbb, pheno_val_ukbb[,c("bmi", "Age_at_recruitment", "Sex")])

        # simplify column names
        colnames(p_mtx_traintest_ukbb)[colnames(p_mtx_traintest_ukbb) == "Age_at_recruitment"] = "age"
        colnames(p_mtx_traintest_ukbb)[colnames(p_mtx_traintest_ukbb) == "Sex"] = "sex"
        colnames(p_mtx_val_ukbb)[colnames(p_mtx_val_ukbb) == "Age_at_recruitment"] = "age"
        colnames(p_mtx_val_ukbb)[colnames(p_mtx_val_ukbb) == "Sex"] = "sex"
        
        # make sex numeric (male = 0, female  = 1)
        p_mtx_traintest_ukbb$sex = ifelse(p_mtx_traintest_ukbb$sex == "Male", 0, 
                                        ifelse(p_mtx_traintest_ukbb$sex == "Female", 1, NA))
        p_mtx_val_ukbb$sex = ifelse(p_mtx_val_ukbb$sex == "Male", 0, 
                                        ifelse(p_mtx_val_ukbb$sex == "Female", 1, NA))
    }

    # remove all proteins, only keep bmi, age and sex
    if(in_pred_mode == "sexagebmi"){
        p_mtx_traintest_ukbb = p_mtx_traintest_ukbb[, c("bmi", "age", "sex")]
        p_mtx_val_ukbb = p_mtx_val_ukbb[, c("bmi", "age", "sex")]
    }

    #check and report
    print("#####################################################")
    print(paste0("traintest: samples in prot and pheno in same order: ", identical(rownames(p_mtx_traintest_ukbb), rownames(pheno_train_test_ukbb)))) # T
    print(paste0("val: samples in prot and pheno in same order:  ", identical(rownames(p_mtx_val_ukbb), rownames(pheno_val_ukbb)))) # T    
    print("#####################################################")
    print(paste0("nr samples in train/test: ", length(unique(rownames(pheno_train_test_ukbb))))) # T
    if(paste0(in_outcome, ".", in_relcol_sub) %in% colnames(pheno_train_test_ukbb)){
        print(paste0("nr cases in train/test: ", sum(pheno_train_test_ukbb[,paste0(in_outcome, ".", in_relcol_sub)] == "1"))) # T
        print(paste0("nr controls in train/test: ", sum(pheno_train_test_ukbb[,paste0(in_outcome, ".", in_relcol_sub)] == "0"))) # T
    }else{
        curr_col_forprinting = colnames(pheno_train_test_ukbb)[grepl(paste0(in_outcome, ".", in_relcol_sub), colnames(pheno_train_test_ukbb))] # T
        print(paste0("nr cases in train/test: ", sum(pheno_train_test_ukbb[,curr_col_forprinting] == "1"))) # T
        print(paste0("nr controls in train/test: ", sum(pheno_train_test_ukbb[,curr_col_forprinting] == "0"))) # T
    }
    print("#####################################################")
    print(paste0("nr samples in val: ", length(unique(rownames(pheno_val_ukbb))))) # T    
    if(paste0(in_outcome, ".", in_relcol_sub) %in% colnames(pheno_val_ukbb)){
        print(paste0("nr cases in train/test: ", sum(pheno_val_ukbb[,paste0(in_outcome, ".", in_relcol_sub)] == "1"))) # T
        print(paste0("nr controls in train/test: ", sum(pheno_val_ukbb[,paste0(in_outcome, ".", in_relcol_sub)] == "0"))) # T
    }else{
        curr_col_forprinting = colnames(pheno_val_ukbb)[grepl(paste0(in_outcome, ".", in_relcol_sub), colnames(pheno_val_ukbb))] # T
        print(paste0("nr cases in train/test: ", sum(pheno_val_ukbb[,curr_col_forprinting] == "1"))) # T
        print(paste0("nr controls in train/test: ", sum(pheno_val_ukbb[,curr_col_forprinting] == "0"))) # T
    }
    #print(paste0("nr cases in val: ", sum(pheno_val_ukbb[,paste0(in_outcome, ".", in_relcol_sub)] == "1")))
    #print(paste0("nr controls in val:", sum(pheno_val_ukbb[,paste0(in_outcome, ".", in_relcol_sub)] == "0")))
    print("#####################################################")

    # return
    return(list(p_mtx_traintest_ukbb = p_mtx_traintest_ukbb, pheno_train_test_ukbb = pheno_train_test_ukbb, p_mtx_val_ukbb = p_mtx_val_ukbb, pheno_val_ukbb = pheno_val_ukbb))
}


get_matrices_allsamples <- function(in_path_datalist, in_relcol_sub, is_bmi_used = F, 
                oa_pheno_here = OA_PHENO, path_cc_here = PATH_CC){
    data_list_controlfromcontrolcolumn = get(load(file = in_path_datalist, verbose = T))

    pheno_all = data_list_controlfromcontrolcolumn[[oa_pheno_here]]$cov
    pheno_all$oa_status = finalise_outcome(in_dat = pheno_all, sub_colname = in_relcol_sub)
    pheno_all$bmi = get_bmi(in_path_cc = path_cc_here, in_dat = pheno_all)

    p_mtx_all = data_list_controlfromcontrolcolumn[[oa_pheno_here]]$p_matrix
    # no balanced:  36598  2131
    # balanced:   16920  2037

    # keep samples with bmi information
    if(is_bmi_used){
        pheno_all = pheno_all[!is.na(pheno_all$bmi), ]
        p_mtx_all = p_mtx_all[rownames(pheno_all), ]
    }
    print(paste0("traintest: samples in prot and pheno in same order: ", identical(rownames(pheno_all), rownames(p_mtx_all))))
    return(list(pheno_all = pheno_all, p_mtx_all = p_mtx_all))
}

get_p_anno <- function(in_p = PATH_ANNO){
    p_anno = read.table(in_p, sep = "\t", header = T)
    nrow(p_anno) == length(unique(p_anno$x.olinkid))# T
    rownames(p_anno) = p_anno$x.olinkid

    return(p_anno)
}


perform_pwas <- function(in_p_mtx, in_pheno){

    # prefilter associated proteins
    print(paste0(" --> train data samples in right order: ", identical(rownames(in_pheno), rownames(in_pheno))))

    design <- model.matrix(~ in_pheno$oa_status)
    fit <- lmFit(t(in_p_mtx), design)
    fit <- eBayes(fit)

    thres = 0.05 / nrow(fit$p.value) # 2.45459e-05
    
    #identical(rownames(fit$p.value), rownames(fit$coefficients)) # T

    tmp = p.adjust(fit$p.value[,'in_pheno$oa_status1'], method = "BH")
    tmp2 = rownames(fit$p.value)
    tmp3 = fit$coefficients[, "in_pheno$oa_status1"]
    tmp4 = fit$p.value[,'in_pheno$oa_status1']
    pwas_df = data.frame(protein = tmp2, effect = tmp3, pval = tmp4, fdr = tmp)

    pwas_df$pass_bonf = (pwas_df$pval < thres)
    pwas_df$pass_fdr = (pwas_df$fdr < 0.05)
    rownames(pwas_df) = pwas_df$protein
    rm(tmp, tmp2, tmp3, tmp4)

    #table(pwas_df$pass_bonf)
    #FALSE  TRUE
    # 1163   874
    #table(pwas_df$pass_fdr)
    #FALSE  TRUE
    #  538  1499

   prot_sig = rownames(pwas_df[pwas_df$pass_fdr,])

    if(length(prot_sig) <= 1){ # not enought fdr signal
        if(length(rownames(pwas_df[pwas_df$pval < 0.05,])) >= 2){ # test whether enought nominal signal
            prot_sig = rownames(pwas_df[pwas_df$pval < 0.05,]) # take nominal signal
            print("Use nomin p < 0.05")
        }else{
            pwas_df = pwas_df[order(pwas_df$pval),]
            prot_sig = rownames(pwas_df)[1:200]    # just take top 200
            print("Use only top 200")
        }
    }else{
        print("Use threshold fdr < 0.05")
    }

    print(paste0("Nr included proteins: ", length(unique(prot_sig))))

    return(list(prot_sig = prot_sig, pwas_sumstats = pwas_df))
}



clean_tmp_train_folder <- function(in_p = PATH_OUT_TMPFILES){
    files_rfcm = list.files(path = in_p, pattern = "rf_cm_", full.names = T)
    files_rfmod = list.files(path = in_p, pattern = "rf_mod_", full.names = T)
    files_rfpred = list.files(path = in_p, pattern = "rf_pred_", full.names = T)

    all(do.call(file.remove, list(files_rfcm)))
    all(do.call(file.remove, list(files_rfmod)))
    all(do.call(file.remove, list(files_rfpred)))
}


define_train_grid <- function(nr_feat, rel_row = NULL){
    mtry = ceiling(sqrt(nr_feat))
    mtry = ceiling(c(mtry*0.6, mtry*0.8, mtry*1, mtry*1.2))

    tuneGrid = data.frame(mtry = mtry, min.node.size = c(1,1,1,1), splitrule = c("gini", "gini", "gini", "gini"))

    if(!is.null(rel_row)){
        tuneGrid = tuneGrid[rel_row, ]   
    }
    print(tuneGrid)

    return(tuneGrid)
}

get_traintest_sample = function(in_folds = folds, in_number_folds, in_test_d_index){
          train_d_index = c(1:in_number_folds)[!(c(1:in_number_folds) %in% in_test_d_index)]
          train_d_samples = unlist(sapply(train_d_index, FUN = function(x){
            in_folds[[x]]
          }))
            
          test_d_samples = in_folds[[in_test_d_index]]

          return(list(train_d_samples = train_d_samples, test_d_samples = test_d_samples))
}

# in_number_iter = TRAIN_FI_RF_NR_ITER
# in_number_folds = TRAIN_FI_RF_NR_ITER
# m_in = m_in
# p_in = p_in
# in_ctrl = ctrl
# in_grid = tuneGrid
# outpath = PATH_OUT_TMPFILES



perform_rf_training <- function(in_number_iter, in_number_folds, m_in, p_in, in_ctrl, in_grid, outpath = PATH_OUT_TMPFILES){
    ####################################################################################################################
    #fold_index = 1
    print(c(1:in_number_iter))
    print(c(1:in_number_folds))
    obj_1 = sapply(c(1:in_number_iter), FUN = function(fold_index){
      # (1) create folds for CV: balanced by sample.type (= case or control)
      print(paste0("Current iteration is ", fold_index))
      set.seed(200 * fold_index)
      folds = createFolds(p_in$oa_status, k = in_number_folds, list = TRUE, returnTrain = F)
      
      folds = lapply(folds, FUN = function(x){
        rownames(m_in)[x]
      })
      ####################################################################################################################
      # (2) define training set and test set for current CV-fold 
      # foreach(test_d_index = 1:number_folds) %dopar%{
      # test_d_index = 1
      obj_2 = sapply(c(1:in_number_folds), FUN = function(test_d_index){ 
        ####
        print(paste0("Current fold is ", fold_index, "---", test_d_index))
        ####
        obj_tmp = get_traintest_sample(in_folds = folds, in_number_folds = in_number_folds, in_test_d_index = test_d_index)
                
        ####
        train_d = m_in[obj_tmp$train_d_samples, ]
        test_d =  m_in[obj_tmp$test_d_samples, ]
        
        train_pheno = p_in[obj_tmp$train_d_samples, ]
        test_pheno =  p_in[obj_tmp$test_d_samples, ]
        ####################################################################################################################
        # check
        print(paste0(" --> test data samples (inner loop) in right order: ", identical(rownames(test_pheno), rownames(test_d))))
        print(paste0(" --> train data samples (inner loop) in right order: ", identical(rownames(train_pheno), rownames(train_d))))
        ####################################################################################################################
        # (4) train and test
        set.seed(400 * fold_index)

        # train model
        model_rf <- caret::train(y = train_pheno$oa_status, x = train_d, method = "ranger", preProcess = NULL, 
                                trControl = in_ctrl, metric = "Accuracy", maximize = T, tuneGrid = in_grid, verbose = T, 
                                importance  = "impurity_corrected")
        ##############################################################################################################
        # predict(model_rf) https://stats.stackexchange.com/questions/226109/how-does-predict-randomforest-estimate-class-probabilities
        #final <- data.frame(actual = test_pheno[,"tissue"], predict(model_rf, newdata = test_d_in, type = "prob"))
        #final$predict <- ifelse(final$G > 0.5, "G", "B")
        #final$predict = factor(final$predict, levels = c("B", "G"))
        ##############################################################################################################
        # RANGER
        final <- data.frame(actual = test_pheno[,"oa_status"], predict = predict(model_rf, newdata = test_d))
        final$predict <- factor(ifelse(final$predict == 0, "control", "case"), levels = c("control", "case"))
        final$actual = factor(ifelse(final$actual == 0, "control", "case"), levels = c("control", "case"))
        ##############################################################################################################
        cm_original <- confusionMatrix(final$predict, final$actual, positive = "case")
        ##############################################################################################################
        # save models and cm and 
        save(cm_original, file = paste0(outpath, "rf_cm_", fold_index, "_", test_d_index,".rda"))
        save(model_rf, file = paste0(outpath, "rf_mod_", fold_index, "_", test_d_index,".rda"))
        save(final, file = paste0(outpath, "rf_pred_", fold_index, "_", test_d_index,".rda"))
    return(paste0(outpath, "rf_cm_", fold_index, "_", test_d_index,".rda"))
      })
      return(T)
    })
  }



get_feat_importance <- function(in_p = PATH_OUT_TMPFILES, in_mtx = t(p_mtx_traintest_ukbb), in_p_anno){

    print("########################################################################")

    # estimate variable importance across models
    filel = list.files(path = in_p, pattern = "rf_mod_", full.names = T) 
    print(paste0("Nr files: ", length(filel))) # 15              

    # get 15 iterations
    filel = sapply(filel, FUN = function(x){
        
        if(as.integer(strsplit(basename(x), "[_|\\.]")[[1]][3] ) <= 100){
            return(x)
            #return(as.integer(strsplit(basename(x), "[_|\\.]")[[1]][3] ))
        }else{
            NA
        }
    })
    filel = filel[!is.na(filel)]    
    print(paste0("Nr models: ", length(filel))) # 15              
    ###############
    # get fr models
    rf_mod__list = lapply(filel,FUN = function(x){
    tmp_cm = get(load(x))
        return(tmp_cm)
    })
                                
    # collect importance dataframes
    importance_list = lapply(rf_mod__list, FUN = function(x){
        # varImp(x, scale=F)                 
        varImp(x, scale = F)$importance
    })                 

                    
    # calc importance mean per protein
    length(rownames(in_mtx)) # 2,037
    prot_mean_0 = do.call(rbind, lapply(rownames(in_mtx), FUN = function(prot){
        curr = sapply(importance_list, FUN = function(imp_df){
            tmp = imp_df[prot, "Overall"]
            return(tmp)
        })
        # print(paste0("prot: ", prot, "; nr not na: ", sum(!is.na(curr))))
        curr = data.frame(prot = prot, imp_mean = mean(curr, na.rm = T), imp_median = 
                    median(curr, na.rm = T), imp_sd = sd(curr, na.rm = T), nr_notna = sum(!is.na(curr)), 
                        stringsAsFactors = F)
        return(curr)
    }))

    # TODO: assuming that negative impurity gnini index means "even less predictive than randomn"
    rownames(prot_mean_0) = prot_mean_0$prot                 
    prot_mean = prot_mean_0[order(prot_mean_0$imp_mean, na.last = T, decreasing = T),] #  order by importance mean                      
    prot_mean$rank = c(1:nrow(prot_mean))
    prot_mean$prot_name = as.character(in_p_anno[prot_mean$prot, "x.names"])
    prot_mean = prot_mean[,c("rank", "prot_name", "prot", "imp_mean", "imp_sd", "imp_median")]

    # table(is.na(prot_mean$imp_mean))
    #FALSE  TRUE
    # 1149   888

    # remove proteins that were never used
    print(paste0("Nr proteins before filtering: ", nrow(prot_mean))) #  2037    6
    prot_mean = prot_mean[!is.na(prot_mean$imp_mean), ]
    print(paste0("Nr proteins after filtering: ", nrow(prot_mean))) #  1149    6

    #´print(paste0(cor.test(prot_mean$imp_mean, prot_mean$imp_median, method = "pearson")))

    return(prot_mean)
}


get_sen_spec_acc = function(in_p = PATH_OUT_TMPFILES){
    # (5) investigate models
    filel = list.files(path = in_p, pattern = "rf_cm_", full.names = T)
    print(paste0("Nr files: ", length(filel))) # 15

    # get 50 iterations
    filel = sapply(filel, FUN = function(x){
        
        if(as.integer(strsplit(basename(x), "[_|\\.]")[[1]][3] ) <= 500){
            return(x)
            #return(as.integer(strsplit(basename(x), "[_|\\.]")[[1]][3] ))
        }else{
            NA
        }
    })
    filel = filel[!is.na(filel)]    
    print(paste0("Nr models: ", length(filel)))
    # 15 models

    cm_original_list = lapply(filel,FUN = function(x){
        tmp_cm = get(load(x))
        return(tmp_cm)
    })
    ##################
    #ACCURACY
    acc_vec = sapply(cm_original_list, FUN = function(x) x$overall["Accuracy"])
    print("##############")
    print("Accuracy")
    print(summary(acc_vec))
    print(paste0("sd: ", sd(acc_vec)))
    ##################################################################################
    #SENSITIVITY
    sen_vec = sapply(cm_original_list, FUN = function(x)  x$byClass["Sensitivity"])
    print("##############")
    print("Sensitivity")
    print(summary(sen_vec))
    print(paste0("sd: ", sd(sen_vec)))
    ##################################################################################
    #SPECIFICITY
    spec_vec = sapply(cm_original_list, FUN = function(x) x$byClass["Specificity"])
    print("##############")
    print("Specificity")
    print(summary(spec_vec))
    print(paste0("sd: ", sd(spec_vec)))
    ##################################################################################
    #PPV
    ppv_vec = sapply(cm_original_list, FUN = function(x) x$byClass[["Pos Pred Value"]])
    print("##############")
    print("PPV")
    print(summary(ppv_vec))
    print(paste0("sd: ", sd(ppv_vec)))
    ##################################################################################
    return(in_p)
}
##################################################################################
# grep top 50 proteins from rank; perform rfe to keep maximum "in_nr_proteins" proteins

run_rfe <- function(in_pheno = pheno_train_test_ukbb, in_m = p_mtx_traintest_ukbb, 
                in_dat = prot_mean, in_nr_proteins = NR_PROTEINS, rfe_number = 3, 
                in_label = OUT_LABEL, in_p = PATH_OUT_MODEL, in_min_number_proteins = 2){
        ##################################################
        print("#################################################################")

        # grep top 50 proteins
        if(nrow(in_dat) >= 100){
            prot_top = in_dat[1:100,]
        }else{
            prot_top = in_dat
        }

        # prepare seeds; input in rfe control (based on https://stackoverflow.com/questions/32290513/making-recursive-feature-elimination-using-caret-parallel-on-windows)
        subsetSizes <- c(in_min_number_proteins:in_nr_proteins)
        set.seed(123)
        seeds <- vector(mode = "list", length = (rfe_number + 1))
        for(i in 1:rfe_number) seeds[[i]] <- sample.int(1000, length(subsetSizes) + 1)
        seeds[[(rfe_number + 1)]] <- sample.int(1000, 1)

        # perform rfe only 
        ctrl <- rfeControl(functions = rfFuncs, method="cv", number = rfe_number, verbose = T, seeds = seeds)

        # customised selectSize function to not use "all predictors" as best model
        customSelectSize <- function(x, metric = "Accuracy", maximize = T) {
            # Get performance per size
            perf <- x
            # Exclude the largest size (1000)
            perf <- perf[perf$Variables < max(perf$Variables), ] 
            
            # Find best performance
            best <- perf[which.max(perf[[metric]]), "Variables"]
            
            # Return the best smaller size


            return(best)
        }
        ctrl$functions$selectSize <- customSelectSize

        #ctrl$functions$selectVar
        #function (y, size)
        #{
        #    finalImp <- ddply(y[, c("Overall", "var")], .(var), function(x) mean(x$Overall,
        #        na.rm = TRUE))
        #    names(finalImp)[2] <- "Overall"
        #    finalImp <- finalImp[order(finalImp$Overall, decreasing = TRUE),
        #        ]
        #    as.character(finalImp$var[1:size])
        #}

        # ? rfe
        # x: A matrix or data frame of predictors for model training. This object must have unique column names. For the recipes method, 
        # ‘x’ is a recipe object.
        # y: a vector of training set outcomes (either numeric or factor)
        # sizes: a numeric vector of integers corresponding to the number of features that should be retained
        # number: Either the number of folds or number of resampling iterations

        # lmFuncs, rfFuncs, treebagFuncs, ldaFuncs, nbFuncs
        # WARNING: other than rfFuncs throws error!!

        # seeds: an optional vector of integers for the size. The vector should have length of ‘length(sizes)+1’
        ##################################################
        print(paste0("Prot matrix and pheno data samples in same order: ", 
                                identical(rownames(in_m), rownames(in_pheno)))) # --> T
     
        tmp = t(as.data.frame(t(in_m[, rownames(prot_top)])))
        dim(tmp) #  31443    30

        #cl <- 5 # trying to parallise rfe 
        set.seed(123)
        rfe_result <- rfe(x = tmp, 
                        y = in_pheno$oa_status, maximize = T, 
                        sizes = c(in_min_number_proteins:in_nr_proteins), rfeControl = ctrl, metric = "Accuracy")
        save(rfe_result, file = paste0(in_p, "rfe.rda"))
        ##################################################
        # print rfe_result summary
        print(rfe_result)
        print(rfe_result$optVariables)
        print("#################################################################")
        ##################################################
        rel_features = rfe_result$optVariables
        ##################################################
        res_label = strsplit(in_label, "_")[[1]]
        tmp = which(grepl("top", res_label))
        res_label[tmp] = paste0(res_label[tmp], "rfekeep", length(rel_features))
        res_label = paste0(res_label, collapse = "_")
        ##################################################
        res_obj = list(rel_features = rel_features, in_label = res_label)
        save(res_obj, file = paste0(in_p, "res_obj_feat.rda"))
        ##################################################
        return(res_obj)
}

getfrom_ranked_proteins <- function(in_dat = prot_mean, in_nr_proteins = NR_PROTEINS){
        in_dat = in_dat[rev(order(in_dat$imp_mean)),]

        in_dat$imp_mean_normalised = in_dat$imp_mean/sum(in_dat$imp_mean)
        in_dat$imp_mean_normalised_cumsum = cumsum(in_dat$imp_mean_normalised)

        in_dat$imp_mean_normalised_cumsum[in_nr_proteins] #  0.2899331

        return(rownames(in_dat)[1:in_nr_proteins])
}


get_predictor_types_from_path <- function(in_p){
    test = strsplit(in_p, "/")
    test = test[[1]][length(test[[1]])]

    if(test == "models"){
        return("prot_only")
    }else if(test == "models_comb"){
        return("prot_and_sexagebmi")
    }else if(test == "models_sexagebmi"){
        return("sexagebmi")
    }else{
        return(NA)
    }
}

get_rfe_from_file <- function(in_file, in_model_feat){
    res = ifelse(grepl("_rfe_", in_file), 
                    paste0("rfe_feat: ", length(unique(in_model_feat))), 
                        paste0("rfe_feat: NA"))
    return(res)
}

write_log_file = function(in_pheno_tt, in_pheno_val, in_feat = model_xgboost$finalModel$xNames, in_log_file = log_file, in_dir = PATH_OUT_MODEL, in_prot_mean = prot_mean, 
                          in_pred_mode = PRED_MODE){
    # open file
    out_complete = paste0(in_dir, in_log_file)

    # define file
    write(paste0("model: ", strsplit(basename(in_log_file), "_")[[1]][2]), 
                                file = out_complete, append = TRUE, sep = "\n")
    write(paste0("nrfeat: ", unique(length(in_feat))), 
                                file = out_complete, append = TRUE, sep = "\n")
    write(paste0("type_feat: ", get_predictor_types_from_path(in_p = in_dir)), 
                                file = out_complete, append = TRUE, sep = "\n")

    # add case control numbers
    write(paste0("nr cases all: ", (sum(in_pheno_tt$oa_status == 1) + sum(in_pheno_val$oa_status == 1))), file = out_complete, append = TRUE, sep = "\n")
    write(paste0("nr controls all: ", (sum(in_pheno_tt$oa_status == 0) + sum(in_pheno_val$oa_status == 0))), file = out_complete, append = TRUE, sep = "\n")
    write(paste0("nr cases train/test: ", sum(in_pheno_tt$oa_status == 1)), file = out_complete, append = TRUE, sep = "\n")
    write(paste0("nr controls train/test: ", sum(in_pheno_tt$oa_status == 0)), file = out_complete, append = TRUE, sep = "\n")
    write(paste0("nr cases validation: ", sum(in_pheno_val$oa_status == 1)), file = out_complete, append = TRUE, sep = "\n")
    write(paste0("nr controls validation: ", sum(in_pheno_val$oa_status == 0)), file = out_complete, append = TRUE, sep = "\n")

    # add rfe info
    write(paste0("rfe: ", grepl("_rfe_", in_log_file)), file = out_complete, append = TRUE, sep = "\n")
    write(get_rfe_from_file(in_file = in_log_file, in_model_feat = in_feat), 
                                file = out_complete, append = TRUE, sep = "\n")

    if(in_pred_mode != "sexagebmi"){
        # add predictors
        write(paste0("############################"), 
                            file = out_complete, append = TRUE, sep = "\n")
        write(paste0("proteins"), 
                            file = out_complete, append = TRUE, sep = "\n")
        write.table(in_prot_mean[in_feat,], file = out_complete, 
                    append = T, , quote = F, sep = "\t", row.names = F, col.names = TRUE)
        write(paste0("############################"), file = out_complete, append = TRUE, sep = "\n")
    }
    return(T)
}

add_performanc_log_file_fromrow <- function(in_row, in_f = log_file){
    # in_f = paste0(PATH_OUT_MODEL, RF_MODEL_LABEL, ".log")
    write(paste0("#########################"), file = in_f, append = TRUE, sep = "\n")
    write(paste0("Sensitivity: ", round(in_row$"Sensitivity", 4), 
                "\nSpecificity: ", round(in_row$"Specificity", 4), 
                "\nBalanced Accuracy: ", round(in_row$"Bal_Acc", 4), 
                "\nPos Pred Value: ", round(in_row$"PPV", 4), 
                "\nAUC (binary): ", round(in_row$"AUC_bin", 4), 
                "\nAUC (probabilities): ", round(in_row$"AUC_prob", 4), 
                "\nAUPRC (binary): ", round(in_row$"PRAUC_bin", 4), 
                "\nAUPRC (probabilities): ", round(in_row$"PRAUC_prob", 4)), file = in_f, append = TRUE, sep = "\n")
    return(T)
}

add_performance_log_file <- function(in_f = log_file, in_cm = cm_original){
    # in_f = paste0(PATH_OUT_MODEL, RF_MODEL_LABEL, ".log")
    write(paste0("#########################"), file = in_f, append = TRUE, sep = "\n")
    write(paste0("Sensitivity: ", round(in_cm$byClass[["Sensitivity"]], 4), 
                "\nSpecificity: ", round(in_cm$byClass[["Specificity"]], 4), 
                "\nBalanced Accuracy: ", round(in_cm$byClass[["Balanced Accuracy"]], 4), 
                "\nPos Pred Value: ", round(in_cm$byClass[["Pos Pred Value"]], 4)), file = in_f, append = TRUE, sep = "\n")
    return(T)
}

##################################################################################
get_roc_curve <- function(curr_model, in_feat_roc, in_test_d, in_pheno = pheno_val_ukbb){
    ################
    roc_df = predict(curr_model, newdata = in_test_d[,in_feat_roc], type = "prob") # get ppedictions
    roc_df = cbind(id = rownames(in_test_d), roc_df, # make a df out of predictions
                no_prob_prediction = factor(as.character(predict(curr_model, newdata = 
                        in_test_d[, in_feat_roc])), levels = c(0,1), ordered = TRUE)) 
    roc_df$actual = in_pheno[rownames(in_test_d),"oa_status"]


    # [TEST] how are these "no_prob_prediction" calies gemerated?
    roc_df = cbind(roc_df, no_prob_bin = 
                        ifelse(as.integer(as.character(roc_df$no_prob_prediction)) < 0.5 , 0 , 1))
    identical(as.character(roc_df$no_prob_prediction), as.character(roc_df$no_prob_bin))
    # T
    # CONCLUSION: take probabilities, threshold at 0.5
    ################ 
    # make roc curve

    rocobj_1 <- roc(response = roc_df$actual, predictor = roc_df$'1', levels=c(0, 1)) # keeping probabilitites
    #Setting levels: control = 0, case = 1
    #Setting direction: controls < cases

    rocobj_2 <- roc(response = roc_df$actual, predictor = roc_df$no_prob_prediction, levels=c(0, 1)) # binarised prediction
    #Setting levels: control = 0, case = 1
    #Setting direction: controls < cases

    #rocobj_test <- roc(response = roc_df$actual, predictor = factor(as.character(rep(1, length(roc_df$actual))), ordered = T, levels = c(0,1)), levels=c(0, 1))
    ################
    # get youdens j to get 

    rocobj_1_youden <- coords(rocobj_1, x = "best", best.method = "youden", ret = "threshold")

    rocobj_2_youden <- coords(rocobj_2, x = "best", best.method = "youden", ret = "threshold")
    ################
    return(list(rocobj_1_probs = rocobj_1, rocobj_2_binarised = rocobj_2, rocobj_1_youden = rocobj_1_youden, rocobj_2_youden = rocobj_2_youden))
    #return(list(rocobj_1 = rocobj_1, rocobj_2 = rocobj_2, rocobj_test = rocobj_test))
}

get_proc_curve <- function(curr_model, in_feat_pr, in_test_d = test_d, in_pheno = pheno_val_ukbb){
    final <- data.frame(actual = in_pheno[,"oa_status"], predict_prob = predict(curr_model, newdata = in_test_d[,in_feat_pr], type = "prob"), 
                        predict_bin = predict(curr_model, newdata = in_test_d[,in_feat_pr]))

    # print auc
    print("##############################")
    print("Binary predictions:")
    print(pr.curve(final[final$actual == 1, "predict_bin"], final[final$actual == 0, "predict_bin"], curve=TRUE))
    print("##############################")
    print("Probabilities:")
    print(pr.curve(final[final$actual == 1, "predict_prob.1"], final[final$actual == 0, "predict_prob.1"], curve=TRUE))

    bin_obj =  pr.curve(final[final$actual == 1, "predict_bin"], final[final$actual == 0, "predict_bin"], curve=TRUE) # --> also good!
    prob_obj = pr.curve(final[final$actual == 1, "predict_prob.1"], final[final$actual == 0, "predict_prob.1"], curve=TRUE) # --> also good!

    return(list(bin = bin_obj, prob = prob_obj))
}

# in_p_ponly = PATH_OUT_MODEL_PONLY
# in_p_sexagebmi = PATH_OUT_MODEL_sexagebmi
# in_p_comb = PATH_OUT_MODEL_COMB
get_paths_notmp <- function(in_pred_mode, in_p_ponly, in_p_sexagebmi, in_p_comb){
    if(in_pred_mode == "prot_only"){
        PATH_OUT_MODEL = in_p_ponly
    }else if(in_pred_mode == "sexagebmi"){
        PATH_OUT_MODEL = in_p_sexagebmi
    }else if(in_pred_mode == "comb"){
        PATH_OUT_MODEL = in_p_comb
    }else{
        stop("No valid mode. set parameter to: prot_only or sexagebmi or comb")
    }
    return(PATH_OUT_MODEL = PATH_OUT_MODEL)
}

get_model_label = function(in_list_here){
    if(length(in_list_here) == 1){ # there is only one model
        return(strsplit(in_list_here[1], "\\.")[[1]][1])
    }else{
        stop(paste0("More than one model --> Do it manually! label: ", in_list_here))
    }
}

adjust_cc_threshold <- function(opt_thresh_model, opt_thresh_test_d, opt_thresh_pheno_val, use_default = F){
        if(!use_default){
            test_here <- do.call(rbind, lapply(seq(0,1,0.1), FUN = function(curr_thresh){
                curr =  data.frame(actual = opt_thresh_pheno_val[,"oa_status"], predict = predict(opt_thresh_model, newdata = opt_thresh_test_d, type = "prob"))
                curr$predict <- ifelse(curr$'predict.1' >= curr_thresh, 1, 0)
                curr$predict <- factor(ifelse(curr$predict == 0, "control", "case"), levels = c("control", "case"))
                curr$actual = factor(ifelse(curr$actual == 0, "control", "case"), levels = c("control", "case"))

                cm_original <- confusionMatrix(curr$predict, curr$actual, positive = "case")

                res_here = data.frame(thresh = curr_thresh, sens = cm_original$byClass[["Sensitivity"]], 
                        spec = cm_original$byClass[["Specificity"]],
                        bal_acc = cm_original$byClass[["Balanced Accuracy"]])
                return(res_here)
            }))
            thresh_output = test_here[which.max(test_here$bal_acc), "thresh"]
        }else{
            thresh_output = 0.5
        }

        # (2) get final resutls
        curr =  data.frame(actual = opt_thresh_pheno_val[,"oa_status"], predict = predict(opt_thresh_model, newdata = opt_thresh_test_d, type = "prob"))
        curr$predict <- ifelse(curr$'predict.1' >= thresh_output, 1, 0)
        curr$predict <- factor(ifelse(curr$predict == 0, "control", "case"), levels = c("control", "case"))
        curr$actual = factor(ifelse(curr$actual == 0, "control", "case"), levels = c("control", "case"))

        cm_original <- confusionMatrix(curr$predict, curr$actual, positive = "case")

        return(list(cm_original = cm_original, final = curr, final_thresh = thresh_output))
}

get_path_to_model <- function(in_pred, in_path_to_pheno){
    if(in_pred == "protein_only"){
        PATH_TO_MODEL = paste0(in_path_to_pheno, "/2_all_strict_noagesex/models_nrfeat30/models_V2/") #model_xgboost_rose_top30rfekeep8_rfe_tunegrid10.rda")
        PATH_TO_MODEL = list.files(PATH_TO_MODEL, ".rda", full.names = T)[1]
    }else if(in_pred == "comb"){
        PATH_TO_MODEL = paste0(in_path_to_pheno, "/2_all_strict_noagesex/models_nrfeat30/models_comb_V2/") #model_xgboost_rose_top30rfekeep8_rfe_tunegrid10.rda")
        PATH_TO_MODEL = list.files(PATH_TO_MODEL, ".rda", full.names = T)[1]
    }
    return(PATH_TO_MODEL)
}

get_features_from_model <- function(in_path_to_model){

    in_model <- get(load(file = in_path_to_model, verbose = T))
    rel_feat = in_model$finalModel$x

    return(rel_feat)
}
################################################################

ERROR: Error in nrow(pheno): object 'pheno' not found
